## Fake News Corpus
### https://github.com/several27/FakeNewsCorpus/releases/tag/v1.0
Mapping: (tag)  ->  label
- (satire)      ->  1-"Satire"
- (conspiracy)  ->  2-"Hoax"
- (bias)        ->  3-"Propaganda"
- (reliable)    ->  4-"Reliable News"

In [1]:
import sys
import csv
import pandas as pd

In [4]:
csv.field_size_limit(500000)

500000

In [5]:
CHUNK_SIZE = 10 ** 6

In [6]:
with pd.read_csv('news_cleaned_2018_02_13.csv', chunksize=CHUNK_SIZE, index_col="id", engine='python') as reader:
    for chunk in reader:
        type_content = chunk[['type', 'content']]
        filtered = type_content[(type_content.type == 'satire') | (type_content.type == 'conspiracy') | (type_content.type == 'bias') | (type_content.type == 'reliable')]
        filtered['type'] = filtered['type'].map({'satire': 1, 'conspiracy': 2, 'bias': 3, 'reliable': 4})
        filtered.to_csv('fakenewscorpus.csv', mode='a', index=False, header=False)

C:\Users\Hong\AppData\Local\Temp\ipykernel_38628\3604349914.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'conspiracy': 2, 'bias': 3, 'reliable': 4})
C:\Users\Hong\AppData\Local\Temp\ipykernel_38628\3604349914.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['type'] = filtered['type'].map({'satire': 1, 'conspiracy': 2, 'bias': 3, 'reliable': 4})
C:\Users\Hong\AppData\Local\Temp\ipykernel_38628\3604349914.py:5: SettingWithCopyWarning: 
A

In [7]:
df = pd.read_csv('fakenewscorpus.csv', header=None, index_col=False)
y = df.iloc[:, 0]
y.value_counts()

0
4    1913222
3    1138998
2     831235
1     112948
Name: count, dtype: int64